In [ ]:
from depmap_omics_upload import tracker as track
import pandas as pd
import dalmatian as dm

In [ ]:
wm = dm.WorkspaceManager("fccredits-silver-tan-7621/CCLE_v2")
ccle_samples = wm.get_samples()

In [ ]:
mytracker = track.SampleTracker()
seq_table = mytracker.add_model_cols_to_seqtable()

In [ ]:
seq_table

In [ ]:
for m in ccle_samples.index:
    if pd.isna(ccle_samples.loc[m, "hg38_wgs_bam"]):
        if len(seq_table[(seq_table.ModelID == m) & (seq_table.expected_type == "wgs")]) > 1:
            cdsid = seq_table[(seq_table.ModelID == m) & (seq_table.expected_type == "wgs") & (seq_table.version == 2)].index.tolist()[0]
            ccle_samples.loc[m, "hg38_wgs_bam"] = seq_table.loc[cdsid, "bam_filepath"]
            ccle_samples.loc[m, "hg38_wgs_bai"] = seq_table.loc[cdsid, "bai_filepath"]
        if len(seq_table[(seq_table.ModelID == m) & (seq_table.expected_type == "wgs")]) == 1:
            cdsid = seq_table[(seq_table.ModelID == m) & (seq_table.expected_type == "wgs")].index.tolist()[0]
            ccle_samples.loc[m, "hg38_wgs_bam"] = seq_table.loc[cdsid, "bam_filepath"]
            ccle_samples.loc[m, "hg38_wgs_bai"] = seq_table.loc[cdsid, "bai_filepath"]

In [ ]:
ccle_samples.loc["ACH-001001", "hg38_wgs_bam"]

In [ ]:
wm.update_sample_attributes(ccle_samples)

In [ ]:
cmds = []

for m in ccle_samples.index:
    if not pd.isna(ccle_samples.loc[m, "hg38_wgs_bam"]):
        cmds.append("gsutil -u broad-firecloud-ccle acl ch -u AllUsers:R " + ccle_samples.loc[m, "hg38_wgs_bam"])
        cmds.append("gsutil -u broad-firecloud-ccle acl ch -u AllUsers:R " + ccle_samples.loc[m, "hg38_wgs_bai"])

In [ ]:
from depmap_omics_upload.mgenepy.utils import helper as h

h.parrun(cmds, cores=8)

In [ ]:
print("done")